# Import Libreries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import ast
import gc
import os
import re
import time
import datetime
from tqdm import tqdm

import spacy
import nltk
from nltk.corpus import stopwords

from transformers import pipeline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
nltk.download('stopwords')

nlp = spacy.load("en_core_web_sm")
stop_words = set(stopwords.words('english'))

# Loading the dataset

In [ ]:
recipe_llm = pd.read_csv('../datasets/ds_verbs.csv')
recipe_llm.head(5)

,title,ingredients,directions,link,source,NER,NER_list,NER_len,directions_len,title_lower,NER_clean,ingredients_str,cluster,verbs,lemmed_verbs,filtered_verbs,set_verbs
0,""" Bar"" Cheese","[""32 ounces Velveeta cheese (regular)"", ""1 1/2...","[""In a large sauce pan over low heat, melt the...",www.food.com/recipe/bar-cheese-42151,Gathered,"[""Velveeta cheese"", ""mayonnaise"", ""horseradish...","[velveeta cheese, mayonnaise, horseradish, tab...",5,368,""" bar"" cheese","[velveeta cheese, mayonnaise, horseradish, tab...",velveeta cheese mayonnaise horseradish tabasco...,1,"[pan, add, stirring, add, stirring, cool, refr...","[pan, add, stir, add, stir, cool, refrigerate]","[pan, add, stir, add, stir, cool, refrigerate]","[add, pan, cool, stir, refrigerate]"
1,""" Barber's"" Chewy Vanilla & Salted Butter Toffee","[""2 14 cups icing sugar"", ""12 cup milk"", ""14 c...","[""Butter a loaf pan."", ""In a pot,combine icing...",www.food.com/recipe/barbers-chewy-vanilla-salt...,Recipes1M,"[""icing sugar"", ""milk"", ""corn syrup"", ""vanilla...","[icing sugar, milk, corn syrup, vanilla bean, ...",5,557,""" barber's"" chewy vanilla & salted butter toffee","[icing sugar, milk, corn syrup, vanilla bean, ...",icing sugar milk corn syrup vanilla bean butter,3,"[pot, stirring, reduce, remove, stirring, reac...","[pot, stir, reduce, remove, stir, reach, pour,...","[pot, stir, reduce, remove, stir, reach, pour,...","[reduce, remove, pot, reach, cut, pour, stir]"
2,""" Denauseating"" With Ginger Tea","[""1 1/2 cups water"", ""1/2 teaspoon of crushed ...","[""Bring water to boil with crushed ginger in i...",www.food.com/recipe/denauseating-with-ginger-t...,Gathered,"[""water"", ""ginger"", ""honey"", ""milk""]","[water, ginger, honey, milk]",4,239,""" denauseating"" with ginger tea","[water, ginger, honey, milk]",water ginger honey milk,3,"[crushed, let, medium, strain, add, add, want]","[crush, let, medium, strain, add, add, want]","[crush, let, medium, strain, add, add, want]","[add, want, crush, strain, let, medium]"
3,""" Dunkin Donuts "" Oreo Coffee Coolatta","[""16 Oreo's - crushed with few ones set sdide""...","[""Add all your ice cubes into the blender."", ""...",cookpad.com/us/recipes/483003-dunkin-donuts-or...,Recipes1M,"[""coffee"", ""heavy cream"", ""chocolate syrup"", ""...","[coffee, heavy cream, chocolate syrup, sugar]",4,503,""" dunkin donuts "" oreo coffee coolatta","[coffee, heavy cream, chocolate syrup, sugar]",coffee heavy cream chocolate syrup sugar,3,"[add, blender, give, using, blend, give, want,...","[add, blender, give, use, blend, give, want, g...","[add, blender, give, use, blend, give, want, g...","[add, blend, want, use, make, give, blender, get]"
4,""" German"" Barbecued Carrots","[""1 lb carrot, peeled and sliced"", ""6 slices b...","[""Preheat oven to 350F."", ""Cook bacon until cr...",www.food.com/recipe/german-barbecued-carrots-1...,Recipes1M,"[""carrot"", ""bacon"", ""tomato soup"", ""sugar""]","[carrot, bacon, tomato soup, sugar]",4,256,""" german"" barbecued carrots","[carrot, bacon, tomato soup, sugar]",carrot bacon tomato soup sugar,3,"[cook, peel, place, baking, add, bake, eat]","[cook, peel, place, bake, add, bake, eat]","[cook, peel, place, bake, add, bake, eat]","[add, cook, eat, place, peel, bake]"


In [ ]:
# Standard pipeline setup that's known to work
pipe = pipeline(
    "text-generation",
    model="mistralai/Mistral-7B-Instruct-v0.2",
    torch_dtype=torch.float16,
    device_map="auto"
)

print(pipe.model.generation_config)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Device set to use cuda:0


GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2
}



### Assigning the dish categories

The following code has been commented out for computational reason. The results can be found in the datasets folder.

In [ ]:
# !huggingface-cli login

# # Memory management helpers

# def clear_memory():
#     """Force garbage collection and clear GPU cache."""
#     gc.collect()
#     if torch.cuda.is_available():
#         torch.cuda.empty_cache()

# def get_gpu_memory_usage():
#     """Return GPU memory usage in GB."""
#     if torch.cuda.is_available():
#         return torch.cuda.memory_allocated() / 1e9
#     return 0

# # Set up the Mistral pipeline
# pipe = pipeline(
#     "text-generation",
#     model="mistralai/Mistral-7B-Instruct-v0.2",
#     torch_dtype=torch.float16,
#     device_map="auto"
# )

# print(f"Current GPU memory usage: {get_gpu_memory_usage():.2f} GB\n")

# # Define the dish type classification function
# def classify_dish_type_with_pipeline(recipe_data):
#     """
#     Get the model's reasoning about the recipe's dish type and it returns the raw model response.
#     """
#     recipe_name = recipe_data.get('title_lower', 'Unknown')
#     verbs = recipe_data.get('filtered_verbs')

#     # Safe parsing of ingredients with ast.literal_eval
#     ingredients = recipe_data.get('NER_list', [])
#     if isinstance(ingredients, str):
#         try:
#             ingredients = ast.literal_eval(ingredients)
#         except (ValueError, SyntaxError):
#             ingredients = [ingredients]

#     prompt = f"""
# <s>[INST]
# Classify this recipe into exactly ONE best-fitting category:
# 0 = Not food
# 1 = Breakfast Bakes, Breads & Pancakes (pancakes, waffles, morning dishes, cornbread)
# 2 = Desserts, Cookies & Cakes (sweet treats, pies, sweet sauces, puddings, custards)
# 3 = Drinks, Punches & Fruit Treats (beverages, smoothies, juices)
# 4 = Salads, Slaws, Dips & Dressings (ALL dips whether hot or cold, spreads, condiments)
# 5 = Pasta & Vegetable Casseroles/Bakes (pasta dishes, stuffings, vegetable bakes)
# 6 = Soups, Stews & One-Pots (ALL soups including fruit soups, chilis, broths)
# 7 = Meat & Fish Mains (protein-centered dishes where meat/fish is central)

# RULES:
# - ALL dips = category 4 even if hot or containing meat
# - ALL soups = category 6 even if fruit-based or containing meat
# - ALL sweet creamy desserts (puddings, custards) = category 2, even if no baking
# - Mixed dishes with both meat and pasta/vegetables go in category 5 if pasta/vegetables dominate, 7 if meat is the focus
# - Breads served as sides (cornbread, hush puppies) = category 1

# Answer in EXACTLY this format (no additions):
# RATIONALE: 3-10 keywords
# NUMBER = X Category-Name

# Recipe: {recipe_name}
# Ingredients: {', '.join(ingredients) if isinstance(ingredients, list) else ingredients}
# Cooking verbs: {verbs}
# [/INST]</s>
# """

#     try:
#         # Using inference_mode for faster processing (safer optimization)
#         with torch.inference_mode():
#             response = pipe(prompt, max_new_tokens=60, do_sample=False)
#         full_text = response[0]['generated_text']
#         return full_text[len(prompt):].strip()
#     except Exception as e:
#         print(f"Error processing '{recipe_name}': {e}")
#         return f"Error: {e}"

# # Batch processing function
# def process_in_batches(df, batch_size=20, checkpoint_every=100):
#     """
#     Process recipes in batches with checkpoint saving and progress tracking.
#     """
    
#     os.makedirs("dish_type_results", exist_ok=True)

#     # Check for existing checkpoints to resume from
#     checkpoint_files = [f for f in os.listdir("dish_type_results") if f.startswith("dish_type_results_checkpoint_")]

#     # Sort checkpoint files by number to find the most recent
#     checkpoint_nums = []
#     for file in checkpoint_files:
#         match = re.search(r'checkpoint_(\d+)\.csv', file)
#         if match:
#             checkpoint_nums.append(int(match.group(1)))

#     # Start from the most recent checkpoint if available
#     results = []
#     start_idx = 0

#     if checkpoint_nums:
#         last_checkpoint = max(checkpoint_nums)
#         checkpoint_file = f"dish_type_results/dish_type_results_checkpoint_{last_checkpoint}.csv"

#         print(f"[{datetime.datetime.now()}] Found checkpoint at {checkpoint_file}. Resuming from there.")
#         temp_df = pd.read_csv(checkpoint_file)

#         # Make sure the dish_type_response column exists
#         if 'dish_type_response' in temp_df.columns:
#             results = temp_df['dish_type_response'].tolist()
#             start_idx = len(results)
#             print(f"[{datetime.datetime.now()}] Loaded {start_idx} existing results. Continuing from recipe {start_idx+1}.")

#     # Process remaining recipes
#     start_time = time.time()
#     total_recipes = len(df)
#     remaining_recipes = total_recipes - start_idx

#     if remaining_recipes <= 0:
#         print(f"[{datetime.datetime.now()}] All recipes already processed. Nothing to do.")
#         return results

#     print(f"[{datetime.datetime.now()}] Starting dish type classification for {remaining_recipes} remaining recipes...")

#     # Calculate batches for remaining recipes
#     total_batches = (remaining_recipes + batch_size - 1) // batch_size

#     for batch_idx in range(total_batches):
#         # Calculate indices for the current batch
#         batch_start = start_idx + (batch_idx * batch_size)
#         batch_end = min(start_idx + ((batch_idx + 1) * batch_size), total_recipes)

#         print(f"\n[{datetime.datetime.now()}] Processing batch {batch_idx + 1}/{total_batches} (recipes {batch_start + 1}-{batch_end})")
#         batch = df.iloc[batch_start:batch_end]

#         batch_results = []
#         for row in tqdm(batch.itertuples(), total=len(batch), desc="Processing recipes"):
#             response = classify_dish_type_with_pipeline({
#                 'title_lower': getattr(row, 'title_lower', 'Unknown'),
#                 'filtered_verbs': getattr(row, 'filtered_verbs', ''),
#                 'NER_list': getattr(row, 'NER_list', [])
#             })
#             batch_results.append(response)

#             clear_memory()

#         # Add batch
#         results.extend(batch_results)
#         recipes_processed = len(results)

#         # Saving
#         if recipes_processed % checkpoint_every == 0 or batch_idx == total_batches - 1:
#             df_temp = df.iloc[:recipes_processed].copy()
#             df_temp['dish_type_response'] = results
#             checkpoint_file = f"dish_type_results/dish_type_results_checkpoint_{recipes_processed}.csv"
#             df_temp.to_csv(checkpoint_file, index=False)
#             print(f"[{datetime.datetime.now()}] Checkpoint saved: {checkpoint_file}")

#         # Display progress statistics
#         elapsed_time = time.time() - start_time
#         recipes_per_second = (recipes_processed - start_idx) / elapsed_time if elapsed_time > 0 else 0

#         # ETA
#         if recipes_per_second > 0:
#             remaining_count = total_recipes - recipes_processed
#             estimated_remaining = remaining_count / recipes_per_second

#             print(f"[{datetime.datetime.now()}] Progress: {recipes_processed}/{total_recipes} recipes ({recipes_processed/total_recipes*100:.1f}%)")
#             print(f"Speed: {recipes_per_second*60:.1f} recipes/minute")
#             print(f"Elapsed time: {elapsed_time/60:.1f} minutes")
#             print(f"Estimated time remaining: {estimated_remaining/60:.1f} minutes")

#         print(f"Current GPU memory usage: {get_gpu_memory_usage():.2f} GB")

#         # Add cooldown period between batches
#         if batch_idx < total_batches - 1:
#             cooldown_time = 5  # 5 seconds
#             print(f"[{datetime.datetime.now()}] Cooling down for {cooldown_time} seconds...")
#             time.sleep(cooldown_time)
#             clear_memory()

#     return results

# # DataFrame
# def main(recipe_df):
#     print(f"[{datetime.datetime.now()}] Starting dish type classification on {len(recipe_df)} recipes...")
#     dish_type_responses = process_in_batches(recipe_df, batch_size=50, checkpoint_every=250)

#     # Append full
#     recipe_df['dish_type_response'] = dish_type_responses
#     final_file = "recipes_with_dish_type_responses.csv"
#     recipe_df.to_csv(final_file, index=False)
#     print(f"\n[{datetime.datetime.now()}] Classification complete! Full responses saved to '{final_file}'.")
#     return recipe_df

# # Execute 
# if __name__ == "__main__":
#     # Load
#     recipe_llm = pd.read_csv('../datasets/ds_verbs.csv')
#     recipe_llm = main(recipe_llm)


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `Colab Access` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Colab

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Device set to use cuda:0


[2025-05-14 21:10:40.710146] Model loaded successfully!
Current GPU memory usage: 14.48 GB

[2025-05-14 21:10:40.947834] Starting dish type classification on 25758 recipes...
[2025-05-14 21:10:40.948997] Found checkpoint at dish_type_results/dish_type_results_checkpoint_25758.csv. Resuming from there.
[2025-05-14 21:10:42.831775] Loaded 25758 existing results. Continuing from recipe 25759.
[2025-05-14 21:10:42.832244] All recipes already processed. Nothing to do.

[2025-05-14 21:10:49.455994] Classification complete! Full responses saved to 'recipes_with_dish_type_responses.csv'.


In [ ]:
recipe_llm1 = pd.read_csv('/gdrive/My Drive/NLP_Fanta_Projactt/RecipeNLG/DATASET/human_labelling.csv')
recipe_llm1.head()

recipe_llm1.drop(columns = ["Unnamed: 4", "HumanLabelling", "1\t Breakfast Bakes, Breads & Pancakes (pancakes, waffles, morning dishes, cornbread)\n2\t Desserts, Cookies & Cakes (sweet treats, pies, sweet sauces, puddings, custards)\n3\t Drinks, Punches & Fruit Treats (beverages, smoothies, juices)\n4\t Salads, Slaws, Dips & Dressings (ALL dips whether hot or cold, spreads, condiments)\n5\t Pasta & Vegetable Casseroles/Bakes (pasta dishes, stuffings, vegetable bakes)\n6\t Soups, Stews & One-Pots (ALL soups including fruit soups, chilis, broths)\n7\t Meat & Fish Mains (protein-centered dishes where meat/fish is central)"], inplace=True)

In [ ]:
recipe_llm1

,Unnamed: 0,title,NER_list,filtered_verbs
0,24311,Virginias Quinoa Salad,['corn' 'black beans' 'red bell peppers' 'grap...,['make' 'include']
1,10718,Grandma Miller'S Dressing,['egg' 'flour' 'sugar' 'vinegar' 'water' 'crea...,['thicken' 'grate' 'cook' 'add' 'serve']
2,24772,Working Woman's Poor Boys,['garlic' 'worcestershire sauce' 'salt' 'tabas...,['cook' 'shred' 'shred']
3,15162,Mediterranean Baked Halibut,['tomatoes' 'onion' 'italian dressing' 'capers...,['bottom' 'bake' 'remain' 'drizzle' 'dress' 'b...
4,15767,Molly's Upside Down Peach Cobbler,['flour' 'baking powder' 'sugar' 'salt' 'milk'...,['bake' 'blend' 'grease' 'bake' 'add' 'reserve...
...,...,...,...,...
995,17835,Pear and Feta Bites Recipe,['endive' 'feta cheese' 'pear' 'lemon juice' '...,['end' 'need' 'leave' 'set' 'coat' 'serve']
996,15054,Maui Sweet Potato Bake,['sweet potatoes' 'coconut milk' 'salt' 'white...,['mash' 'test' 'season' 'grease' 'bake' 'prehe...
997,22421,Sunny's Bourbon Whipped Cream,['heavy cream' 'bourbon' 'sugar'],['form' 'add' 'continue' 'whisk' 'hold' 'whisk...
998,5881,Cinnamon And Blueberry Oatmeal,['milk' 'rolled oats' 'brown sugar' 'ground ci...,['medium' 'stir' 'reduce' 'bowl' 'serve' 'top']


In [ ]:
!huggingface-cli login

# ------------------ Step 1: Memory Management Helpers ------------------ #
def clear_memory():
    """Force garbage collection and clear GPU cache."""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

def get_gpu_memory_usage():
    """Return GPU memory usage in GB."""
    if torch.cuda.is_available():
        return torch.cuda.memory_allocated() / 1e9
    return 0

# ------------------ Step 2: Set Up the Mistral Pipeline ------------------ #
print(f"[{datetime.datetime.now()}] Loading Mistral model via pipeline...")

# Standard pipeline setup that's known to work
pipe = pipeline(
    "text-generation",
    model="mistralai/Mistral-7B-Instruct-v0.2",
    torch_dtype=torch.float16,
    device_map="auto"
)

print(f"[{datetime.datetime.now()}] Model loaded successfully!")
print(f"Current GPU memory usage: {get_gpu_memory_usage():.2f} GB\n")

# ------------------ Step 3: Define the Dish Type Classification Function ------------------ #
def classify_dish_type_with_pipeline(recipe_data):
    """
    Get the model's complete reasoning about the recipe's dish type.
    Returns the raw model response.
    """
    recipe_name = recipe_data.get('title_lower', 'Unknown')
    verbs = recipe_data.get('filtered_verbs')

    # Safe parsing of ingredients with ast.literal_eval
    ingredients = recipe_data.get('NER_list', [])
    if isinstance(ingredients, str):
        try:
            ingredients = ast.literal_eval(ingredients)
        except (ValueError, SyntaxError):
            ingredients = [ingredients]

    # Improved prompt with scratchpad approach
    prompt = f"""
<s>[INST]
Classify this recipe into exactly ONE best-fitting category:
1 = Breakfast Bakes, Breads & Pancakes (pancakes, waffles, morning dishes, cornbread)
2 = Desserts, Cookies & Cakes (sweet treats, pies, sweet sauces, puddings, custards)
3 = Drinks, Punches & Fruit Treats (beverages, smoothies, juices)
4 = Salads, Slaws, Dips & Dressings (ALL dips whether hot or cold, spreads, condiments)
5 = Pasta & Vegetable Casseroles/Bakes (pasta dishes, stuffings, vegetable bakes)
6 = Soups, Stews & One-Pots (ALL soups including fruit soups, chilis, broths)
7 = Meat & Fish Mains (protein-centered dishes where meat/fish is central)

RULES:
- ALL dips = category 4 even if hot or containing meat
- ALL soups = category 6 even if fruit-based or containing meat
- ALL sweet creamy desserts (puddings, custards) = category 2, even if no baking
- Mixed dishes with both meat and pasta/vegetables go in category 5 if pasta/vegetables dominate, 7 if meat is the focus
- Breads served as sides (cornbread, hush puppies) = category 1

ALWAYS OUTPUT EXACTLY THIS (no deviation):
RATIONALE: <your reasoning here>
NUMBER = <digit 0–7> <Category-Name>

Recipe: {recipe_name}
Ingredients: {', '.join(ingredients) if isinstance(ingredients, list) else ingredients}
Cooking verbs: {verbs}
[/INST]</s>
"""

    try:
        # Using inference_mode for faster processing (safer optimization)
        with torch.inference_mode():
            response = pipe(prompt, max_new_tokens=400, do_sample=False)
        full_text = response[0]['generated_text']
        return full_text[len(prompt):].strip()
    except Exception as e:
        print(f"Error processing '{recipe_name}': {e}")
        return f"Error: {e}"

# ------------------ Step 4: Batch Processing Function ------------------ #
def process_in_batches(df, batch_size=20, checkpoint_every=100):
    """
    Process recipes in batches with checkpoint saving and progress tracking.
    """
    # Create results directory if it doesn't exist
    os.makedirs("dish_type_results", exist_ok=True)

    # Check for existing checkpoints to resume from
    checkpoint_files = [f for f in os.listdir("dish_type_results") if f.startswith("dish_type_results_checkpoint_")]

    # Sort checkpoint files by number to find the most recent
    checkpoint_nums = []
    for file in checkpoint_files:
        match = re.search(r'checkpoint_(\d+)\.csv', file)
        if match:
            checkpoint_nums.append(int(match.group(1)))

    # Start from the most recent checkpoint if available
    results = []
    start_idx = 0

    if checkpoint_nums:
        last_checkpoint = max(checkpoint_nums)
        checkpoint_file = f"dish_type_results/dish_type_results_checkpoint_{last_checkpoint}.csv"

        print(f"[{datetime.datetime.now()}] Found checkpoint at {checkpoint_file}. Resuming from there.")
        temp_df = pd.read_csv(checkpoint_file)

        # Make sure the dish_type_response column exists
        if 'dish_type_response' in temp_df.columns:
            results = temp_df['dish_type_response'].tolist()
            start_idx = len(results)
            print(f"[{datetime.datetime.now()}] Loaded {start_idx} existing results. Continuing from recipe {start_idx+1}.")

    # Process remaining recipes
    start_time = time.time()
    total_recipes = len(df)
    remaining_recipes = total_recipes - start_idx

    if remaining_recipes <= 0:
        print(f"[{datetime.datetime.now()}] All recipes already processed. Nothing to do.")
        return results

    print(f"[{datetime.datetime.now()}] Starting dish type classification for {remaining_recipes} remaining recipes...")

    # Calculate batches for remaining recipes
    total_batches = (remaining_recipes + batch_size - 1) // batch_size

    for batch_idx in range(total_batches):
        # Calculate indices for the current batch
        batch_start = start_idx + (batch_idx * batch_size)
        batch_end = min(start_idx + ((batch_idx + 1) * batch_size), total_recipes)

        print(f"\n[{datetime.datetime.now()}] Processing batch {batch_idx + 1}/{total_batches} (recipes {batch_start + 1}-{batch_end})")
        batch = df.iloc[batch_start:batch_end]

        batch_results = []
        # Using itertuples instead of iterrows for better performance
        for row in tqdm(batch.itertuples(), total=len(batch), desc="Processing recipes"):
            response = classify_dish_type_with_pipeline({
                'title_lower': getattr(row, 'title_lower', 'Unknown'),
                'filtered_verbs': getattr(row, 'filtered_verbs', ''),
                'NER_list': getattr(row, 'NER_list', [])
            })
            batch_results.append(response)

            # Clear memory after each recipe (original behavior)
            clear_memory()

        # Add batch results to overall results
        results.extend(batch_results)
        recipes_processed = len(results)

        # Save checkpoint
        if recipes_processed % checkpoint_every == 0 or batch_idx == total_batches - 1:
            df_temp = df.iloc[:recipes_processed].copy()
            df_temp['dish_type_response'] = results
            checkpoint_file = f"dish_type_results/dish_type_results_checkpoint_{recipes_processed}.csv"
            df_temp.to_csv(checkpoint_file, index=False)
            print(f"[{datetime.datetime.now()}] Checkpoint saved: {checkpoint_file}")

        # Display progress statistics
        elapsed_time = time.time() - start_time
        recipes_per_second = (recipes_processed - start_idx) / elapsed_time if elapsed_time > 0 else 0

        # Calculate estimated time remaining
        if recipes_per_second > 0:
            remaining_count = total_recipes - recipes_processed
            estimated_remaining = remaining_count / recipes_per_second

            print(f"[{datetime.datetime.now()}] Progress: {recipes_processed}/{total_recipes} recipes ({recipes_processed/total_recipes*100:.1f}%)")
            print(f"Speed: {recipes_per_second*60:.1f} recipes/minute")
            print(f"Elapsed time: {elapsed_time/60:.1f} minutes")
            print(f"Estimated time remaining: {estimated_remaining/60:.1f} minutes")

        print(f"Current GPU memory usage: {get_gpu_memory_usage():.2f} GB")

        # Add cooldown period between batches
        if batch_idx < total_batches - 1:
            cooldown_time = 5  # 5 seconds
            print(f"[{datetime.datetime.now()}] Cooling down for {cooldown_time} seconds...")
            time.sleep(cooldown_time)
            clear_memory()

    return results

# ------------------ Step 5: Process the DataFrame ------------------ #
def main(recipe_df):
    print(f"[{datetime.datetime.now()}] Starting dish type classification on {len(recipe_df)} recipes...")
    dish_type_responses = process_in_batches(recipe_df, batch_size=50, checkpoint_every=250)

    # Append full responses to the dataframe and save final results
    recipe_df['dish_type_response'] = dish_type_responses
    final_file = "recipes_with_dish_type_responses.csv"
    recipe_df.to_csv(final_file, index=False)
    print(f"\n[{datetime.datetime.now()}] Classification complete! Full responses saved to '{final_file}'.")
    return recipe_df

# Execute if this is the main script
if __name__ == "__main__":
    # Load your recipe dataframe
    recipe_llm = recipe_llm1
    recipe_llm = main(recipe_llm)


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `Colab Access` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Colab

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

...

[2025-05-23 18:52:21.805527] Progress: 900/1000 recipes (90.0%)
Speed: 12.9 recipes/minute
Elapsed time: 69.7 minutes
Estimated time remaining: 7.7 minutes
Current GPU memory usage: 14.49 GB
[2025-05-23 18:52:21.805887] Cooling down for 5 seconds...

[2025-05-23 18:52:27.144122] Processing batch 19/20 (recipes 901-950)


Processing recipes: 100%|██████████| 50/50 [03:48<00:00,  4.58s/it]


[2025-05-23 18:56:16.142739] Progress: 950/1000 recipes (95.0%)
Speed: 12.9 recipes/minute
Elapsed time: 73.6 minutes
Estimated time remaining: 3.9 minutes
Current GPU memory usage: 14.49 GB
[2025-05-23 18:56:16.142994] Cooling down for 5 seconds...

[2025-05-23 18:56:21.475102] Processing batch 20/20 (recipes 951-1000)


Processing recipes: 100%|██████████| 50/50 [03:40<00:00,  4.40s/it]


[2025-05-23 19:00:01.612496] Checkpoint saved: dish_type_results/dish_type_results_checkpoint_1000.csv
[2025-05-23 19:00:01.612578] Progress: 1000/1000 recipes (100.0%)
Speed: 12.9 recipes/minute
Elapsed time: 77.4 minutes
Estimated time remaining: 0.0 minutes
Current GPU memory usage: 14.49 GB

[2025-05-23 19:00:02.027782] Classification complete! Full responses saved to 'recipes_with_dish_type_responses.csv'.
